In [1]:
from geolifeclef.utils import get_spark

spark = get_spark()
df = spark.read.parquet(
    "../../data/geolifeclef-2024/data/processed/tiles/pa-train/BioClimatic_Average_1981-2010/bio1/v3"
)
df.printSchema()
print(f"Number of rows: {df.count()}")
df.show(n=1, vertical=True, truncate=80)

24/05/24 15:22:45 WARN Utils: Your hostname, daphne-major resolves to a loopback address: 127.0.1.1; using 172.28.199.217 instead (on interface eth0)
24/05/24 15:22:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/24 15:22:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/24 15:22:46 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


root
 |-- bio1: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- surveyId: double (nullable = true)
 |-- target: long (nullable = true)

Number of rows: 1483637
-RECORD 0------------------------------------------------------------------------------------
 bio1     | [-7840.45751953125, 1401.9239501953125, -354.29833984375, -18.208463668823242... 
 lat      | 50.88698                                                                         
 lon      | 5.824586                                                                         
 surveyId | 3895400.0                                                                        
 target   | 140                                                                              
only showing top 1 row



In [2]:
df = spark.read.parquet(
    "../../data/geolifeclef-2024/data/processed/tiles/pa-train/BioClimatic_Average_1981-2010/bio19/v3"
)
df.printSchema()

root
 |-- bio19: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- surveyId: double (nullable = true)
 |-- target: long (nullable = true)



In [3]:
df = spark.read.parquet(
    "../../data/geolifeclef-2024/data/processed/dct_timeseries/combined_timeseries_v4"
)
df.printSchema()

root
 |-- surveyId: integer (nullable = true)
 |-- TimeSeries: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- ts_blue: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- ts_green: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- ts_nir: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- ts_red: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- ts_swir1: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- ts_swir2: array (nullable = true)
 |    |-- element: float (containsNull = true)



In [5]:
from pyspark.sql import functions as F

df.select(F.countDistinct("surveyId")).show()

+------------------------+
|count(DISTINCT surveyId)|
+------------------------+
|                 3919661|
+------------------------+



In [6]:
len(df.first().ts_red)

64

In [7]:
df.select(
    *[F.array_size(x) for x in df.columns if x.startswith("ts_")]
).describe().show()

24/05/24 15:23:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------------+--------------------+------------------+------------------+--------------------+--------------------+
|summary|array_size(ts_blue)|array_size(ts_green)|array_size(ts_nir)|array_size(ts_red)|array_size(ts_swir1)|array_size(ts_swir2)|
+-------+-------------------+--------------------+------------------+------------------+--------------------+--------------------+
|  count|            5010983|             5010983|           5010983|           5010983|             5010983|             5010983|
|   mean|               64.0|                64.0|              64.0|              64.0|                64.0|                64.0|
| stddev|                0.0|                 0.0|               0.0|               0.0|                 0.0|                 0.0|
|    min|                 64|                  64|                64|                64|                  64|                  64|
|    max|                 64|                  64|                64|              

In [9]:
(
    df.withColumn("is_null", F.array_size("ts_red").isNull())
    .groupBy("is_null")
    .agg(F.countDistinct("surveyId"))
).show()

+-------+------------------------+
|is_null|count(DISTINCT surveyId)|
+-------+------------------------+
|  false|                 3919661|
+-------+------------------------+

